# DAT-DEN-03 | Demo 14 | Latent Variable Modeling with `gensim`

`gensim` (http://radimrehurek.com/gensim) is a library of language processing tools focused on latent variable models of text.

In [ ]:
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import feature_extraction
from gensim import matutils, models

pd.set_option('display.max_rows', 10)
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 10)

%matplotlib inline
plt.style.use('ggplot')

The data is about sentiments on Amazon reviews.

In [ ]:
reviews = []
sentiments = []

with open(os.path.join('..', 'datasets', 'amazon-reviews.txt')) as f:
    for line in f.readlines():
        line = line.strip('\n')
        review, sentiment = line.split('\t')
        sentiment = np.nan if sentiment == '' else int(sentiment)

        reviews.append(review.lower())
        sentiments.append(sentiment)

df = pd.DataFrame({'review': reviews, 'sentiment': sentiments})

In [ ]:
df.head()

In [ ]:
df.dropna(inplace = True) # Let's drop the NaN

In [ ]:
df.head()

## LDA with `gensim`

### Let's first translate a set of documents (articles) into a matrix representation with a row per document and a column per feature (word or n-gram)

In [ ]:
vectorizer = feature_extraction.text.CountVectorizer(stop_words = 'english')

In [ ]:
documents = vectorizer.fit_transform(df.review)

In [ ]:
# Let's now build a mapping of numerical ID to word

id2word = dict(enumerate(vectorizer.get_feature_names()))

In [ ]:
id2word

### We want to learn which columns are correlated (i.e., likely to come from the same topic).  This is the word distribution.  We can also determine what topics are in each document, the topic distribution.

In [ ]:
# First we convert our word-matrix into gensim's format

corpus = matutils.Sparse2Corpus(documents, documents_columns = False)

(Check https://radimrehurek.com/gensim/matutils as needed)

In [ ]:
corpus

(Check https://radimrehurek.com/gensim/models/ldamodel as needed)

In [ ]:
# Then we fit an LDA model

model = models.ldamodel.LdaModel(corpus = corpus, num_topics = 25, id2word = id2word, passes = 10)

In this model, we need to explicitly specify the number of topic we want the model to uncover.  This is a critical parameter, but there isn't much guidance on how to choose it.  Try to use domain expertise where possible.

In [ ]:
model

### Goodness of fit

Now we need to assess the goodness of fit for our model.  Like other unsupervised learning techniques, our validation techniques are mostly about interpretation.

Use the following questions to guide you:
- Did we learn reasonable topics?
- Do the words that make up a topic make sense?
- Is this topic helpful towards our goal?

In [ ]:
model.print_topics()

Some topics will be clearer than others.  The following topics represent clear concepts:
- Cooking and Recipes: 0.009 \* cup + 0.009 \* recipe + 0.007 \* make + 0.007 \* food + 0.006 \* sugar
- Cooking and recipes: 0.013 \* butter + 0.010 \* baking + 0.010 \* dough + 0.009 \* cup + 0.009 \* sugar
- Fashion and Style: 0.013 \* fashion + 0.006 \* like + 0.006 \* dress + 0.005 \* style

## Word2Vec with `gensim`

In [ ]:
# Setup the body text
sentences = df.review.map(lambda review: review.split())

In [ ]:
sentences

In [ ]:
model = models.Word2Vec(sentences, size = 100, window = 5, min_count = 5, workers = 4)

`Word2Vec` has many arguments:
- `size` represents how many concepts or topics we should use
- `window` represents how many words surrounding a sentence we should use as our original feature
- `min_count` is the number of times that context or word must appear
- `workers` is the number of CPU cores to use to speed up model training

(Check http://radimrehurek.com/gensim/models/word2vec as needed)

In [ ]:
model

### Most similar words

The model has a `most_similar` function that helps find the words most similar to the one you queried.  This will return words that are most often used in the same context.

In [ ]:
model.most_similar(positive = ['great'])

In [ ]:
vectorizer.get_feature_names()

In [ ]:
sentences

In [ ]:
sentences = list(map(lambda sentence: list(filter(lambda word: word in vectorizer.get_feature_names(), sentence)), sentences))

In [ ]:
sentences

In [ ]:
model = models.Word2Vec(sentences, size = 100, window = 5, min_count = 5, workers = 4)

In [ ]:
model.most_similar(positive = ['great'])